In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.gaia import Gaia
from astroquery.vizier import Vizier
from astropy.table import Table
Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source" # Select early Data Release 3

In [4]:
path = 'C:\\Users\\gamak\\Documents\\Papers\\Stars\\'

In [5]:
xr =pd.read_csv(path+'gaia_xstars_pm_r98_200pc_lkh10_woutdup.gzip')

In [6]:
catalog_list = Vizier.find_catalogs('Ritter')
print({k:v.description for k,v in catalog_list.items()})

{'III/265': 'RAVE 3rd data release (Siebert+, 2011)', 'III/272': 'RAVE 4th data release (Kordopatis+, 2013)', 'III/283': 'RAVE 6th data release (Steinmetz+, 2020)', 'V/59': 'Cataclysmic Binaries, LMXBs and related objects, 5th Ed. (Ritter 1990)', 'V/82': 'Cataclysmic binaries (Ritter+ 1995)', 'V/99': 'Cataclysmic Binaries and LMXB Catalogue (Ritter+ 1998)', 'V/110': 'Catalog of Cataclysmic Variables (Downes+ 2001)', 'V/113D': 'Cataclysmic Binaries, LMXBs, and related objects (Ritter+, 2003)', 'V/123A': 'Catalog of Cataclysmic Variables (Downes+ 2001-2006)', 'B/cb': 'Cataclysmic Binaries, LMXBs, and related objects (Ritter+, 2004)', 'J/ApJ/851/107': 'iPTF 16asu photometry follow-up (Whitesides+, 2017)', 'J/ApJS/225/24': 'NuGrid stellar data set I. Yields from H to Bi (Pignatari+, 2016)', 'J/A+AS/123/273': 'Illumination in binaries (Hameury+, 1997)', 'J/AJ/117/1313': 'NGC 6441 VI photometry (Layden+, 1999)'}


In [8]:
result = Vizier.get_catalogs('B/cb')

In [9]:
print(result)

TableList with 10 tables:
	'0:B/cb/cbdata' with 11 column(s) and 50 row(s) 
	'1:B/cb/lmxbdata' with 11 column(s) and 50 row(s) 
	'2:B/cb/pcbdata' with 11 column(s) and 50 row(s) 
	'3:B/cb/findrefs' with 3 column(s) and 50 row(s) 
	'4:B/cb/cbrefs' with 3 column(s) and 50 row(s) 
	'5:B/cb/lmxbrefs' with 3 column(s) and 50 row(s) 
	'6:B/cb/pcbrefs' with 3 column(s) and 50 row(s) 
	'7:B/cb/whoswho1' with 3 column(s) and 50 row(s) 
	'8:B/cb/whoswho2' with 3 column(s) and 50 row(s) 
	'9:B/cb/whoswho5' with 2 column(s) and 50 row(s) 


In [10]:
catalog = SkyCoord(ra=xr.ra.values*u.degree, dec=xr.dec.values*u.degree)

In [11]:
coord = SkyCoord(ra=result[2]['RAJ2000'], dec=result[2]['DEJ2000'], unit = (u.hourangle, u.deg))

In [12]:
max_sep = 3.0 * u.arcsec
idx, d2d, d3d = coord.match_to_catalog_3d(catalog)
sep_constraint = d2d < max_sep
c_matches = coord[sep_constraint]
catalog_matches = catalog[idx[sep_constraint]]
catalog_matches

<SkyCoord (ICRS): (ra, dec) in deg
    [(294.91189321, -6.06390463)]>

<SkyCoord (ICRS): (ra, dec) in deg
    []>

In [ ]:
for i in range(xr.shape[0]):
    result = Vizier.query_region(SkyCoord(ra=xr.ra.iloc[i], dec=xr.dec.iloc[i],
                                             unit=(u.deg, u.deg),
                                             frame='icrs'),
                         radius=3*u.arcsecond,
                         catalog='B/cb')
    if str(result) == 'Empty TableList':
        ind.append(False)
    else:
        ind.append(True)

In [56]:
ind

[False, False]

In [16]:
print(SkyCoord(ra=result['B/cb/cbdata']['RAJ2000'][0], dec=result['B/cb/cbdata']['DEJ2000'][0],unit=(u.hourangle, u.deg)))

<SkyCoord (ICRS): (ra, dec) in deg
    (274.05541667, 49.86777778)>


In [18]:
    coord = SkyCoord(ra=result['B/cb/cbdata']['RAJ2000'][0], dec=result['B/cb/cbdata']['DEJ2000'][0],unit=(u.hourangle, u.deg), frame='icrs')
    radius = u.Quantity(2.0, u.arcsecond)
    j = Gaia.cone_search_async(coord, radius)
    r = j.get_results()
    r.pprint()

TimeoutError: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера